In [1]:
import numpy as np
import math
import pandas as pd
from collections import Counter
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

In [24]:
#Parâmetros do usuário (Definir entrada dos dados)
estat_ord = 'min' #Estatística para ordenamento
qtd_alg_nd = 1 #Quantidade máxima de algoritmos para separar o conjunto de possíves não-duplicadas
k = 4 #Tamanho da janela
orcamento = 100 #Ou uma porcentagem da base de dados
seed = 500

In [3]:
def geraF1(toClass, classificador): #toClass é o conjunto de treinamento
    
    f1 = 0
    
    if classificador == 'SVM':
        modelo = SVC(random_state = seed)
    elif classificador == 'DT':
        modelo = DecisionTreeClassifier(random_state = seed)
    
    #Separação do conjunto X e y
    XtoClass = toClass.iloc[:,1:]
#     print(XtoClass)
    ytoClass = toClass.duplicata
    
    #Divisão dos conjuntos de treino e teste (20% para esse último)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(XtoClass, ytoClass, test_size=0.20, random_state=seed)
    
    #Validação cruzada
    #This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.
    kfold = StratifiedKFold(n_splits=5, random_state=seed) #n_splits igual ao tamanho do conjunto de treino?
    cv_results = model_selection.cross_val_score(modelo, X_train, y_train, cv=kfold, scoring='f1')
    f1 = cv_results.mean()
    std = cv_results.std()
    coef_var = std/f1
#     tamTreino = len(XtoClass)
    
    return f1, std, coef_var#, tamTreino

In [4]:
#geraTrainSet(weight_vector_dict_orig, dirDest, 'train' + '(' + algUtl + ')' + num + '.csv')

def geraTrainSet(ct, dir, file1):


    #Desnecessária essa parte se quiser deixar a primeira coluna com o status das duplicatas
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
    ct.to_csv(dir+file1, sep=';', index=False)     
    
#geraTestSet(weight_vector_dict_orig, dirDest, 'test' + '(' + algUtl + ')' + num + '.csv')

def geraTestSet(ct, dir, file1):


    #Desnecessária essa parte se quiser deixar a primeira coluna com o status das duplicatas
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
    ct.to_csv(dir+file1, sep=';', index=False)            

In [5]:
#Leitura do conjunto de pares conflitantes
pc = "../../csv/conjuntosDS/NAO_DA3.csv"
# pc = pd.read_csv(pc, sep=';') #pares conflitantes
pc = pd.read_csv(pc, sep=';', index_col=['elemento1', 'elemento2']) #pares conflitantes
# pc

In [6]:
pc_aa = pc.iloc[:, :5 ] #Conjunto onde serão aplicadas as janelas deslizantes
pc_vetores = pc.iloc[:, 5: ] #Conjunto base para compor o conjunto treinamento 

# pc_aa = pc.loc[:, :'med' ]
# pc_vetor = pc.loc[:, 'med': ] #Como referenciar a coluna vizinha à 'med'?

duplicata = pc_aa.loc[:, 'duplicata' ]

#Adicionando a coluna de duplicatas a pc_vetores
pc_vetores = pd.concat([duplicata, pc_vetores], axis=1, ignore_index=False)

In [7]:
pc_aa.head()

,,duplicata,qtdAlg,min,max,med
elemento1,elemento2,,,,,
4975,5889,1.0,21,0.903782,1.000000,0.963424
5577,8990,1.0,20,0.942708,1.000000,0.974635
3475,8592,1.0,19,0.910085,0.980903,0.958294
3192,7923,1.0,20,0.950000,1.000000,0.979358
4395,8725,1.0,20,0.944444,0.984375,0.963168


In [8]:
pc_vetores.head()

,,duplicata,title,artist,track01,track02,track03,track10,track11
elemento1,elemento2,,,,,,,,
4975,5889,1.0,1.000000,1.0,1.000000,0.833333,1.000000,0.833333,1.000000
5577,8990,1.0,1.000000,1.0,1.000000,0.937500,1.000000,0.937500,1.000000
3475,8592,1.0,0.966667,1.0,0.888889,1.000000,0.857143,1.000000,0.857143
3192,7923,1.0,1.000000,1.0,0.900000,1.000000,1.000000,1.000000,1.000000
4395,8725,1.0,1.000000,1.0,0.888889,0.933333,1.000000,0.933333,1.000000


In [9]:
#Separação do conjunto de pares conflitantes em dois a partir da quantidade de algoritmos 
#que aponta o par como possível duplicata 
#(conjunto A com quantidade de algoritmos = 1, com maioria composta por possíveis não-duplicatas,
#e conjunto B com quantidade de algoritmos > 1, contendo mais possíveis duplicatas que o conjunto A)
ndup = pc_aa.loc[pc['qtdAlg'] == qtd_alg_nd]
dup = pc_aa.loc[pc['qtdAlg'] != qtd_alg_nd]

#Ordenamento dos pares pela estatística selecionada
ndup = ndup.sort_values(estat_ord) 
dup = dup.sort_values(estat_ord, ascending=False)

In [10]:
ndup.head()

,,duplicata,qtdAlg,min,max,med
elemento1,elemento2,,,,,
104289,105349,0.0,1,0.488872,0.488872,0.488872
102831,105349,0.0,1,0.492994,0.492994,0.492994
105349,105518,0.0,1,0.496958,0.496958,0.496958
103997,105349,0.0,1,0.504825,0.504825,0.504825
101480,107779,0.0,1,0.506742,0.506742,0.506742


In [11]:
dup.head()

,,duplicata,qtdAlg,min,max,med
elemento1,elemento2,,,,,
7008,8831,1.0,22,1.0,1.0,1.0
4481,6716,1.0,22,1.0,1.0,1.0
2004,3710,1.0,10,1.0,1.0,1.0
7201,8536,1.0,19,1.0,1.0,1.0
4384,5657,1.0,20,1.0,1.0,1.0


In [12]:
len(dup) + len(ndup)

1101

In [13]:
#Criação do dataframe que armazenará o conjunto de treinamento

conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)
# conj_treino.insert(0,'duplicata', None)
conj_treino

,duplicata,title,artist,track01,track02,track03,track10,track11


In [25]:
#Povoamento inicial sem aleatoriedade no except

import sys

orcamento = 100
conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)

continua = True
f1_anterior = 0
f1_atual = 0
f1_svm = 0
f1_dt = 0

jan_inic_ndup = 0 
jan_fin_ndup = k

jan_inic_dup = 0 
jan_fin_dup = k

cont = 0

entrouExcept = False


#Povoamento inicial com 20 pares rotulados (10 de cada)
while cont < 10:

    print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_ndup: {2}".format(orcamento, jan_inic_dup, jan_fin_ndup))
    
    #Desliza-se a janela
    foi = True
    
    while foi:
        grupo = ndup.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe

        #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
        #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

        try:

            id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada
            foi = False
            orcamento -= 1

        except ValueError:

            print("Atualizando janelas no except")
            jan_inic_ndup = jan_inic_ndup + k
            jan_fin_ndup = jan_fin_ndup + k
            
            foi = True
            
        except:
            print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores")

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    
    print("Possível não-duplicata")
    print(vetor_sel)
    
    foi = True
    
    #SELEÇÃO DA POSSÍVEL DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    
    while foi:
    
        grupo = dup.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe

        try:

            id_row_sel = grupo['qtdAlg'].idxmin() #Retorna o índice da linha com maior valor da coluna ('min') especificada
            foi = False
            orcamento -= 1

        except ValueError:

            print("Atualizando janelas no except")
            jan_inic_dup = jan_inic_dup + k
            jan_fin_dup = jan_fin_dup + k
            
            foi = True

        except:
            print('Eita!')
        
    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    print("Possível duplicata")
    print(vetor_sel)
    
    #Atualização das janelas
    jan_inic_ndup = jan_inic_ndup + k
    jan_fin_ndup = jan_fin_ndup + k
    
    jan_inic_dup = jan_inic_dup + k
    jan_fin_dup = jan_fin_dup + k
    
    
    cont += 1
    
    
#Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM')
f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT')
    
f1_atual = (f1_svm + f1_dt)/2
std_atual = (std_svm + std_dt)/2
cv_atual = (cv_svm + cv_dt)/2

orcamento: 100 - jan_inic_dup: 0 - jan_fin_ndup: 4
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.500000
track02      0.916667
track03      0.818182
track10      0.916667
track11      0.818182
Name: (103997, 105349), dtype: float64
Possível duplicata
duplicata    1.0
title        1.0
artist       0.0
track01      1.0
track02      1.0
track03      1.0
track10      1.0
track11      1.0
Name: (2004, 3710), dtype: float64
orcamento: 98 - jan_inic_dup: 4 - jan_fin_ndup: 8
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.625000
track02      0.181818
track03      0.416667
track10      0.181818
track11      0.416667
Name: (101480, 103830), dtype: float64
Possível duplicata
duplicata    1.0
title        1.0
artist       0.0
track01      1.0
track02      1.0
track03      1.0
track10      1.0
track11      1.0
Name: (1858, 9637), dtype: float64
orcamento: 96 - jan_inic_dup: 8 - jan_fin_n

In [27]:
#Estatísticas do povoamento inicial

print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

print("F1: {0}".format(f1_atual))

print("Desvio padrão: {0}".format(std_atual))

cv_atual = cv_atual*100
print("Coeficiente de variação: {0}%.".format(str(cv_atual)[:5]))

duplicatas = [i for i in conj_treino.duplicata if i == True]
percentage = float(duplicatas.count(True)/float(len(conj_treino))*100)
print("{0}% de duplicatas.".format(str(percentage)[:5]))

naoduplicatas = [i for i in conj_treino.duplicata if i == False]
percentage2 = float(naoduplicatas.count(False)/float(len(conj_treino))*100)
print("{0}% de nao-duplicatas.".format(str(percentage2)[:5]))   

print("orcamento: {0} - jan_inic_ndup: {1} - jan_fin_ndup: {2}".format(orcamento, jan_inic_ndup, jan_fin_ndup))
print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_dup: {2}".format(orcamento, jan_inic_dup, jan_fin_dup))

Quantidade de itens no conjunto de treinamento: 20
F1: 0.96
Desvio padrão: 0.07999999999999999
Coeficiente de variação: 833.3%.
50.0% de duplicatas.
50.0% de nao-duplicatas.
orcamento: 80 - jan_inic_ndup: 40 - jan_fin_ndup: 44
orcamento: 80 - jan_inic_dup: 40 - jan_fin_dup: 44


In [28]:
#Aplicação das janelas deslizantes após o povoamento incial sem aleatoriedade no except

import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

continua = True
max_local = 0
igual = 0

print("Início")
print("orcamento: {0}".format(orcamento))
print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

while continua:
    
    f1_anterior = f1_atual

    foi = True
    
    while foi:
        grupo = ndup.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe

        #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
        #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação

        try:

            id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada
            foi = False
            orcamento -= 1

        except ValueError:

            print("Atualizando janelas no except")
            jan_inic_ndup = jan_inic_ndup + k
            jan_fin_ndup = jan_fin_ndup + k
            
            foi = True
            
        except:
            print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores")

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    
    print("Possível não-duplicata")
    print(vetor_sel)
    
    foi = True
    
    #SELEÇÃO DA POSSÍVEL DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    
    while foi:
    
        grupo = dup.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe

        try:

            id_row_sel = grupo['qtdAlg'].idxmin() #Retorna o índice da linha com maior valor da coluna ('min') especificada
            foi = False
            orcamento -= 1

        except ValueError:

            print("Atualizando janelas no except")
            jan_inic_dup = jan_inic_dup + k
            jan_fin_dup = jan_fin_dup + k
            
            foi = True

        except:
            print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    
    print("Possível duplicata")
    print(vetor_sel)
    
    #Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
    f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM')
    f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT')

    f1_atual = (f1_svm + f1_dt)/2
    std_atual = (std_svm + std_dt)/2
    cv_atual = (cv_svm + cv_dt)/2 #Coeficiente de variação
    
    cv_atual = cv_atual*100
    
    
    if orcamento > 0 : #Se ainda tem orçamento pra gastar
        if cv_atual < 10:
            if (f1_atual >= f1_anterior) and (igual < 3): #Selecionam-se novos pares para rotulação
    #             print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))
    #             print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

                if f1_atual == f1_anterior:
                    igual += 1
                    print("igual: {0}".format(igual))
                elif (f1_atual > f1_anterior) and (igual > 0):
                    igual = 0

                continua = True
    #             break
            else: #TODO ERRADO! AJEITAR!
                max_local += 1
                #Se entrar aqui, tem que remover os últimos carinhas
                #que entraram no conjunto de treinamento

                #Acho que esses devem ser extraídos para o conjunto PM, não apenas descartados
                conj_treino.drop(conj_treino.tail(2).index,inplace=True)

    #             print("Antes do max_local")

    #             print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))
    #             print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

                print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                if(max_local <= 1):
                    f1_atual = f1_anterior
    #                 print("Depois do max_local")
    #                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                else:
                    f1_atual = f1_anterior
    #                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                    continua = False
                    break
                #Pensar a respeito: Máximo local, máximo global
                #Se a medida de qualidade for igual à anterior (por duas vezes?)
    else:
        continua = False
        break
    
    #Atualização das janelas
    jan_inic_ndup = jan_inic_ndup + k
    jan_fin_ndup = jan_fin_ndup + k
    
    jan_inic_dup = jan_inic_dup + k
    jan_fin_dup = jan_fin_dup + k
    
    print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
    print("igual: {0}".format(igual))

Início
orcamento: 80
Quantidade de itens no conjunto de treinamento: 20
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.240000
track02      0.714286
track03      0.454545
track10      0.714286
track11      0.454545
Name: (100558, 104289), dtype: float64
Possível duplicata
duplicata    1.000000
title        0.975000
artist       1.000000
track01      1.000000
track02      1.000000
track03      0.823529
track10      1.000000
track11      0.823529
Name: (6740, 9193), dtype: float64
F1 anterior: 0.96 - F1 atual: 1.0
igual: 0
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.041667
track02      0.500000
track03      0.550000
track10      0.500000
track11      0.550000
Name: (108429, 3710), dtype: float64
Possível duplicata
duplicata    1.0
title        1.0
artist       1.0
track01      1.0
track02      0.9
track03      1.0
track10      0.9
track11      1.0
Name: (9421, 9489), dtype:

In [30]:
#Salvando os arquivos de treino e teste

dirDest = "./"

geraTrainSet(conj_treino, dirDest, 'train.csv')

indicesCT = conj_treino.index.values.tolist()

geral = pd.concat([pc_vetores,conj_treino]) #Concatenando pc_vetores e conj_treino

#Resta para compor o conjunto teste tudo aquilo que está em pc_vetores, mas não em conj_treino
conj_teste = geral.drop(indicesCT, axis='rows') 

geraTestSet(conj_teste, dirDest, 'test.csv')

In [29]:
#Estatísticas

print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

print("F1: {0}".format(f1_atual))

print("Desvio padrão: {0}".format(std_atual))

print("Coeficiente de variação: {0}%.".format(str(cv_atual)[:5]))

duplicatas = [i for i in conj_treino.duplicata if i == True]
percentage = float(duplicatas.count(True)/float(len(conj_treino))*100)
print("{0}% de duplicatas.".format(str(percentage)[:6]))

naoduplicatas = [i for i in conj_treino.duplicata if i == False]
percentage2 = float(naoduplicatas.count(False)/float(len(conj_treino))*100)
print("{0}% de nao-duplicatas.".format(str(percentage2)[:6]))

print("orcamento: {0} - jan_inic_ndup: {1} - jan_fin_ndup: {2}".format(orcamento, jan_inic_ndup, jan_fin_ndup))
print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_dup: {2}".format(orcamento, jan_inic_dup, jan_fin_dup))

F1 anterior: 1.0 - F1 atual: 1.0
Quantidade de itens no conjunto de treinamento: 28
F1: 1.0
Desvio padrão: 0.0
Coeficiente de variação: 0.0%.
50.0% de duplicatas.
50.0% de nao-duplicatas.
orcamento: 68 - jan_inic_ndup: 60 - jan_fin_ndup: 64
orcamento: 68 - jan_inic_dup: 60 - jan_fin_dup: 64


# Geração do conjunto de treinamento com aleatoriedade na escolha

In [19]:
#Povoamento inicial

import sys

orcamento = 100
conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)

continua = True
f1_anterior = 0
f1_atual = 0
f1_svm = 0
f1_dt = 0

jan_inic_dup = 0 
jan_fin_dup = k

jan_inic_ndup = 0 
jan_fin_ndup = k

cont = 0

entrouExcept = False

#Povoamento inicial com 20 pares rotulados (10 de cada)
while cont < 10:

    print("orcamento: {0} - jan_inic_dup: {1} - jan_fin_ndup: {2}".format(orcamento, jan_inic_dup, jan_fin_ndup))
    
    #Desliza-se a janela
    grupo = ndup.iloc[jan_inic_dup:jan_fin_dup] # three rows of dataframe

    #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    
    try:
    
        id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1
    
    except ValueError:
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1
    except:
        print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores")

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    
    print("Possível não-duplicata")
    print(vetor_sel)
    
    #SELEÇÃO DA POSSÍVEL DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    
    grupo = dup.iloc[jan_inic_ndup:jan_fin_ndup] # three rows of dataframe
    
    try:
    
        id_row_sel = grupo['qtdAlg'].idxmin() #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1

    except ValueError:
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1

    except:
        print('Eita!')
        
    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    print("Possível duplicata")
    print(vetor_sel)
    
    #Atualização da janela
    jan_inic_dup = jan_inic_dup + k
    jan_fin_ndup = jan_fin_ndup + k
    
    
    cont += 1
    
    
#Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM')
f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT')
    
f1_atual = (f1_svm + f1_dt)/2
std_atual = (std_svm + std_dt)/2
cv_atual = (cv_svm + cv_dt)/2

orcamento: 100 - jan_inic_dup: 0 - jan_fin_ndup: 3
Possível não-duplicata
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.500000
track02      0.454545
track03      0.818182
track10      0.454545
track11      0.818182
Name: (105349, 105518), dtype: float64
Possível duplicata
duplicata    1.0
title        1.0
artist       0.0
track01      1.0
track02      1.0
track03      1.0
track10      1.0
track11      1.0
Name: (2004, 3710), dtype: float64
orcamento: 98 - jan_inic_dup: 3 - jan_fin_ndup: 6


ValueError: a must be greater than 0

In [22]:
#Aplicação das janelas deslizantes após o povoamento incial

import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

continua = True
max_local = 0
igual = 0

print("Início")
print("orcamento: {0}".format(orcamento))
print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

while continua:
    
    f1_anterior = f1_atual

    #Desliza-se a janela
    grupo = ndup.iloc[jan_inic:jan_fin] # three rows of dataframe

    #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    try:
    
        id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1
    
    except ValueError:
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1
        
    except:
        print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    
    print("Possível não-duplicata")
    print(vetor_sel)
    
    #SELEÇÃO DA POSSÍVEL DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    grupo = dup.iloc[jan_inic:jan_fin] # three rows of dataframe
    
    try:
    
        id_row_sel = grupo['qtdAlg'].idxmin() #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1

    except ValueError:
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
        orcamento -= 1

    except:
        print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    
    print("Possível duplicata")
    print(vetor_sel)
    
    #Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
    f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM')
    f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT')

    f1_atual = (f1_svm + f1_dt)/2
    std_atual = (std_svm + std_dt)/2
    cv_atual = (cv_svm + cv_dt)/2 #Coeficiente de variação
    
    cv_atual = cv_atual*100
    
    
    if orcamento > 0 : #Se ainda tem orçamento pra gastar
        if cv_atual < 10:
            if (f1_atual >= f1_anterior) and (igual < 3): #Selecionam-se novos pares para rotulação
    #             print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))
    #             print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

                if f1_atual == f1_anterior:
                    igual += 1
                    print("igual: {0}".format(igual))
                elif (f1_atual > f1_anterior) and (igual > 0):
                    igual = 0

                continua = True
    #             break
            else: #TODO ERRADO! AJEITAR!
                max_local += 1
                #Se entrar aqui, tem que remover os últimos carinhas
                #que entraram no conjunto de treinamento

                #Acho que esses devem ser extraídos para o conjunto PM, não apenas descartados
                conj_treino.drop(conj_treino.tail(2).index,inplace=True)

    #             print("Antes do max_local")

    #             print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))
    #             print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

                print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                if(max_local <= 1):
                    f1_atual = f1_anterior
    #                 print("Depois do max_local")
    #                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                else:
                    f1_atual = f1_anterior
    #                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                    continua = False
                    break
                #Pensar a respeito: Máximo local, máximo global
                #Se a medida de qualidade for igual à anterior (por duas vezes?)
    else:
        continua = False
        break
    
    #Atualização da janela
    jan_inic = jan_inic + k
    jan_fin = jan_fin + k
    
    print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
    print("igual: {0}".format(igual))

Início
orcamento: 98
Quantidade de itens no conjunto de treinamento: 2


NameError: name 'jan_inic' is not defined